In [1]:
import os
import nibabel as nib
import numpy as np
from scipy.ndimage.interpolation import zoom
import scipy as sp
from tqdm import tqdm
from tqdm.notebook import tqdm_notebook

import keras
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Reshape, Flatten, LeakyReLU, Dropout, Embedding, Concatenate
from keras.layers.core import Activation
from keras.layers.convolutional import Conv3D, Deconv3D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.normalization import BatchNormalization
from keras.utils.vis_utils import plot_model
from keras.optimizers import Adam


def load_nifti(file_path, mask=None, z_factor=None, remove_nan=False):
    """Load a 3D array from a NIFTI file."""
    img = nib.load(file_path)
    struct_arr = np.array(img.get_data())

    if remove_nan:
        struct_arr = np.nan_to_num(struct_arr)
    if mask is not None:
        struct_arr *= mask
    if z_factor is not None:
        struct_arr = np.around(zoom(struct_arr, z_factor), 0)

    return struct_arr


def save_nifti(file_path, struct_arr):
    """Save a 3D array to a NIFTI file."""
    img = nib.Nifti1Image(struct_arr, np.eye(4))
    nib.save(img, file_path)

Using TensorFlow backend.
/mnt/hdd1/lxc-hdd1/tahjid/anaconda3/envs/tf-gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/mnt/hdd1/lxc-hdd1/tahjid/anaconda3/envs/tf-gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/mnt/hdd1/lxc-hdd1/tahjid/anaconda3/envs/tf-gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint1

In [2]:
def prepareData(use_smooth = False, runningOnServer = True):
    rootDir = 'C:/Users/Eshan/Google Drive UALBERTA/Data/' if not runningOnServer else '/mnt/hdd1/lxc-hdd1/tahjid/PD Data/'
    patientList, patientNumbers, dataset = [], [], []
    labelMap = dict(Control=0, PD=1)
    typeMap = dict(FullScan=0, GrayMatter=1, WhiteMatter=2)
    fullScanPath = rootDir + 'FinalData/'
    wmgmpath = rootDir + 'FinalDataWMGM/' if not use_smooth else rootDir + 'FinalDataWMGMSmooth/'
    prefix = 'mwp' if not use_smooth else 'smwp'
    ext = '.nii'
    for i in tqdm(['Control', 'PD']):
        path = fullScanPath + i + '/'
        listOfFiles = [f for f in os.listdir(path) if f.endswith(ext)]
        for file in tqdm(listOfFiles):
            patientNumbers.append(file[:4])
        path = wmgmpath + i + '/'
        listOfFiles = [f for f in os.listdir(path) if f.endswith(ext)]
        for file in tqdm(listOfFiles):
            filename = file[4:8] if not use_smooth else file[5:9]
            if filename not in patientNumbers:
                continue
            if not use_smooth:
                patientList.append([i, file[4:8]])
            else:
                patientList.append([i, file[5:9]])

    for i in tqdm(patientList):
        path = fullScanPath + i[0] + '/'
        patientIdVal = i[1]
        fullScanvalue = os.path.join(path + i[1] + ext)
        path = wmgmpath + i[0] + '/'
        gmval = os.path.join(path + prefix + str(typeMap['GrayMatter']) + i[1] + ext)
        wmval = os.path.join(path + prefix + str(typeMap['WhiteMatter']) + i[1] + ext)
        labelval = labelMap[i[0]]
        dataset.append([patientIdVal,fullScanvalue, gmval, wmval, labelval])
    return np.array(dataset)
dataset = prepareData()

100%|██████████| 299/299 [00:00<00:00, 1271903.55it/s]

100%|██████████| 299/299 [00:00<00:00, 1299582.28it/s]

100%|██████████| 1170/1170 [00:00<00:00, 307303.88it/s]


In [4]:
from sklearn.model_selection import train_test_split
X = dataset[:,:4]
y = dataset[:,4:]

In [36]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=.20, random_state=42)


In [32]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, data, labels, batch_size=2, dim1=(91,109,91), dim2=(242,145,121) , n_channels=1,
                 n_classes=2, shuffle=True):
        'Initialization'
        self.dim1 = dim1
        self.dim2 = dim2
        self.batch_size = batch_size
        self.labels = list(labels)
        self.data = data
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.list_IDs = list(data[:,:1])
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))
    
    def __numbatches__(self):
        return int(np.floor(len(self.list_IDs) / self.__len__()))
    
    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        # print(indexes)
        # Find list of IDs
        # list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(indexes)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __data_generation(self, indexes):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
#         fs = np.empty((self.batch_size, *self.dim1, self.n_channels))
#         gm = np.empty((self.batch_size, *self.dim2, self.n_channels))
#         wm = np.empty((self.batch_size, *self.dim2, self.n_channels))
        gmwm = np.empty((self.batch_size, *self.dim2, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, count in enumerate(indexes):
            val = self.data[count]
#             fullscan = load_nifti(val[1])
            graymatter = load_nifti(val[2])
            whitematter = load_nifti(val[3])
            # print(fullscan.shape, graymatter.shape, whitematter.shape)
            # break
            # Store sample
#             fs[i,] = fullscan[..., np.newaxis]
#             gm[i,] = graymatter[..., np.newaxis]
#             wm[i,] = whitematter[..., np.newaxis]
#             print(graymatter.shape)
#             print(whitematter.shape)
            gmwm[i,] = np.concatenate((graymatter, whitematter))[..., np.newaxis]


            # Store class
            y[i] = self.labels[i]

        return gmwm, keras.utils.to_categorical(y, num_classes=self.n_classes)
    

In [33]:
d = DataGenerator(X, y)


In [35]:
d.__numbatches__()

2

In [18]:
a = d.__getitem__(0)
# b = np.concatenate((a[1][0].reshape(121,145,121), a[1][0].reshape(121,145,121)), axis=0)

/mnt/hdd1/lxc-hdd1/tahjid/anaconda3/envs/tf-gpu/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
/mnt/hdd1/lxc-hdd1/tahjid/anaconda3/envs/tf-gpu/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
/mnt/hdd1/lxc-hdd1/tahjid/anaconda3/envs/tf-gpu/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which 

In [39]:
def gen(phase_train=True, n_classes = 2 ,params={'z_size':500, 'strides':(11,5,11), 'kernel_size':(22,29,11)}):
    """
    Returns a Generator Model with input params and phase_train
    Args:
        phase_train (boolean): training phase or not
        params (dict): Dictionary with model parameters
    Returns:
        model (keras.Model): Keras Generator model
    """

    z_size = params['z_size']
    strides = params['strides']
    kernel_size = params['kernel_size']
    
    in_label = Input(shape=(1,))
    li = Embedding(n_classes, 50)(in_label)
    li = Dense(z_size)(li)
    li = Reshape((1,1,1,z_size))(li)
    
    
    inputs = Input(shape=(1, 1, 1, z_size))
    
    
    merge_inputs = Concatenate()([inputs, li])
    
    g1 = Deconv3D(filters=1024, kernel_size=kernel_size,
                  strides=(1, 1, 1), kernel_initializer='glorot_normal',
                  bias_initializer='zeros', padding='valid')(merge_inputs)
    g1 = BatchNormalization()(g1, training=phase_train)
    g1 = Activation(activation='relu')(g1)

    g2 = Deconv3D(filters=512, kernel_size=kernel_size,
                  strides=strides, kernel_initializer='glorot_normal',
                  bias_initializer='zeros', padding='same')(g1)
    g2 = BatchNormalization()(g2, training=phase_train)
    g2 = Activation(activation='relu')(g2)

    g3 = Deconv3D(filters=264, kernel_size=kernel_size,
                  strides=(1, 1, 1), kernel_initializer='glorot_normal',
                  bias_initializer='zeros', padding='same')(g2)
    g3 = BatchNormalization()(g3, training=phase_train)
    g3 = Activation(activation='relu')(g3)

    g4 = Deconv3D(filters=128, kernel_size=kernel_size,
                  strides=(1, 1, 1), kernel_initializer='glorot_normal',
                  bias_initializer='zeros', padding='same')(g3)
    g4 = BatchNormalization()(g4, training=phase_train)
    g4 = Activation(activation='relu')(g4)

    g5 = Deconv3D(filters=1, kernel_size=kernel_size,
                  strides=(1, 1, 1), kernel_initializer='glorot_normal',
                  bias_initializer='zeros', padding='same')(g4)
    g5 = BatchNormalization()(g5, training=phase_train)
    g5 = Activation(activation='sigmoid')(g5)

    model = Model(inputs=[inputs, in_label], outputs=g5)
    model.summary()

    return model

In [ ]:
model = generator()
# model.summary()
# plt.show()
plot_model(model, to_file = "generator_plot.png", show_shapes = True, show_layer_names = True)
# 

In [40]:
def dis(phase_train = True, n_classes = 2, params={'cube_len':64, 'strides':(11,5,11), 'kernel_size':(22,29,11), 'leak_value':0.2}):
    """
    Returns a Discriminator Model with input params and phase_train 
    Args:
        phase_train (boolean): training phase or not
        params (dict): Dictionary with model parameters    
    Returns:
        model (keras.Model): Keras Discriminator model
    """
    cube_len = params['cube_len']
    strides = params['strides']
    kernel_size = params['kernel_size'] 
    leak_value = params['leak_value']
    
    in_label = Input(shape=(1,))
    li = Embedding(n_classes, 50)(in_label)
    n_nodes = 242 * 145 * 141
    li = Dense(n_nodes)(li)
    li = Reshape((242, 145, 141, 1))(li)
    
    inputs = Input(shape=(242, 145, 141, 1))
    
    merge_inputs = Concatenate()([inputs, li])
    
    d1 = Conv3D(filters=128, kernel_size=kernel_size,
                  strides=strides, kernel_initializer='glorot_normal',
                  bias_initializer='zeros', padding='same')(merge_inputs)
    d1 = BatchNormalization()(d1, training=phase_train)
    d1 = LeakyReLU(leak_value)(d1)

    d2 = Conv3D(filters=256, kernel_size=kernel_size,
                  strides=strides, kernel_initializer='glorot_normal',
                  bias_initializer='zeros', padding='same')(d1)
    d2 = BatchNormalization()(d2, training=phase_train)
    d2 = LeakyReLU(leak_value)(d2)

    d3 = Conv3D(filters=512, kernel_size=kernel_size,
                  strides=strides, kernel_initializer='glorot_normal',
                  bias_initializer='zeros', padding='same')(d2)
    d3 = BatchNormalization()(d3, training=phase_train)
    d3 = LeakyReLU(leak_value)(d3)

    d4 = Conv3D(filters=1024, kernel_size=kernel_size,
                  strides=strides, kernel_initializer='glorot_normal',
                  bias_initializer='zeros', padding='same')(d3)
    d4 = BatchNormalization()(d4, training=phase_train)
    d4 = LeakyReLU(leak_value)(d4)

    d5 = Conv3D(filters=1, kernel_size=(1,1,1),
                  strides=(1, 1, 1), kernel_initializer='glorot_normal',
                  bias_initializer='zeros', padding='valid')(d4)
    d5 = BatchNormalization()(d5, training=phase_train)
    d5 = Activation(activation='sigmoid')(d5) 
    
    d6 = Flatten()(d5)
    d6 = Dropout(0.4)(d6)
    d6 = Dense(1, activation='sigmoid')(d6)
    model = Model(inputs=[inputs, in_label], outputs=d6)
    model.summary()

    return model

In [23]:
dis = discriminator()
plot_model(dis, to_file = "discriminator_plot.png", show_shapes = True, show_layer_names = True)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 50)        100         input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 1, 4947690)   252332190   embedding_1[0][0]                
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 242, 145, 141 0                                            
__________________________________________________________________________________________________
reshape_1 

In [24]:
def GAN(generator, discriminator):
    model = Sequential()
    model.add(generator)
    discriminator.trainable = False
    model.add(discriminator)
    return model

In [28]:
n_epochs   = 10000
batch_size = 30
g_lr       = 0.008
d_lr       = 0.000001
beta       = 0.5
z_size     = 500
obj_ratio  = 0.5

In [41]:
def train():
    discriminator = dis()
    generator = gen()
    discriminator_on_generator = GAN(generator, discriminator)

    g_optim = Adam(lr=g_lr, beta_1=beta)
    generator.compile(loss='binary_crossentropy', optimizer="SGD")

    d_optim = Adam(lr=d_lr, beta_1=0.9)
    discriminator_on_generator.compile(loss='binary_crossentropy', optimizer=g_optim)
    discriminator.trainable = True
    discriminator.compile(loss='binary_crossentropy', optimizer=d_optim)
  
     
    z_sample = np.random.normal(0, 0.33, size=[batch_size, 1, 1, 1, z_size]).astype(np.float32)
#     volumes = d.getAll(obj=obj, train=True, is_local=is_local, obj_ratio=obj_ratio)
    dataloader = DataGenerator(X, y)
    print('Data loaded .......')
#     volumes = volu/mes[...,np.newaxis].astype(np.float) 

#     if not os.path.exists(train_sample_directory):
#         os.makedirs(train_sample_directory)
#     if not os.path.exists(model_directory):
#         os.makedirs(model_directory)         

    for epoch in trange(n_epochs):
        for batch in trange(dataloader.__numbatches__()):
            batchdata = dataloader.__getitem__(batch)
            print(batchdata[1].shape)
#             idx = np.random.randint(len(volumes), size=batch_size)
#             x = volumes[idx]
#             z = np.random.normal(0, 0.33, size=[batch_size, 1, 1, 1, z_size]).astype(np.float32)

#             generated_volumes = generator.predict(z, verbose=0)

#             X = np.concatenate((x, generated_volumes))
#             Y = np.reshape([1]*batch_size + [0]*batch_size, (-1,1,1,1,1))

#             d_loss = discriminator.train_on_batch(X, Y)       
#             print("d_loss : %f" % (d_loss))

#             z = np.random.normal(0, 0.33, size=[batch_size, 1, 1, 1, z_size]).astype(np.float32)            
#             discriminator.trainable = False
#             g_loss = discriminator_on_generator.train_on_batch(z, np.reshape([1]*batch_size, (-1,1,1,1,1)))
#             discriminator.trainable = True

#             print("g_loss : %f" % (g_loss))

#             if epoch % 1000 == 10:
#                 generator.save_weights(model_directory +'generator_' + str(epoch), True)
#                 discriminator.save_weights(model_directory +'discriminator_' + str(epoch), True)

#             if epoch % 500 == 10:
#                 generated_volumes = generator.predict(z_sample, verbose=0)
#                 generated_volumes.dump(train_sample_directory+'/'+str(epoch))  
train()        

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 1, 50)        100         input_3[0][0]                    
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 1, 4947690)   252332190   embedding_2[0][0]                
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 242, 145, 141 0                                            
__________________________________________________________________________________________________
reshape_2 

ValueError: Layer model_3 expects 2 inputs, but it received 1 input tensors. Input received: [<tf.Tensor 'model_3_input:0' shape=(?, 1) dtype=float32>]